In [1]:
import json
import numpy as np
from pprint import pprint
from os import listdir
from os.path import isfile, join
from MongoOps import MongoOps
from ICSParser import ICSParser
from make_directions_matrix import DirectionsMatrix
import random

In [2]:
locations = []
mops = MongoOps()
sample_data_files = [f for f in listdir("/data") if isfile(join("/data", f))]
for open_house_file in sample_data_files:
    parser = ICSParser("/data/%s" % open_house_file)
    event = parser.to_dict()
    # print(event)
    result = mops.safe_query_for_location_info(event)
    locations += [result]

In [3]:
def random_combination(iterable, r):
    '''
    This function helps test different scenarios.
    '''
    pool = tuple(iterable)
    n = len(pool)
    indices = sorted(random.sample(range(n), r))
    return tuple(pool[i] for i in indices)

random_locations = random_combination(locations, 5)

In [4]:
DM = DirectionsMatrix(random_locations, mops)

In [5]:
DM.generate_simplified_directions_matrix()

In [6]:
sdm = DM.simplified_directions_matrix

In [7]:
# Showing off EST/EDT time of day of the open houses and the conversion to minutes from midnight that day
for i in range(len(sdm)):
    start = int(sdm[i]['dtstart'][9:-3])-600
    start_minutes = 6./10*start
    end = int(sdm[i]['dtend'][9:-3])-600
    end_minutes = 6./10*end
    print('%d (%f), %d (%f)' % (start, start_minutes, end, end_minutes))
    start = sdm[i]['start'] = start
    start_minutes = sdm[i]['start_minutes'] = start_minutes
    end = sdm[i]['end'] = end
    end_minutes = sdm[i]['end_minutes'] = end_minutes

1100 (660.000000), 1400 (840.000000)
900 (540.000000), 1100 (660.000000)
1200 (720.000000), 1400 (840.000000)
1000 (600.000000), 1200 (720.000000)
1100 (660.000000), 1300 (780.000000)


In [125]:
vertices = []
V = None
for i in range(len(sdm)):
    V = {'ID' : i, 
               'start' : sdm[i]['start_minutes'], 
               'end' : sdm[i]['end_minutes'], 
               'edges' : sdm[i]['durations'],
               'address_hash' : sdm[i]['address_hash']}
    vertices += [V]

In [126]:
def been_visited(visited, v):
    '''
    Checks to make sure you're not going back to a node that has already been visited.
    '''
    return v in visited

In [127]:
""" 
A Python Class
A simple Python graph class, demonstrating the essential 
facts and functionalities of graphs.
Original implementation from https://www.python-course.eu/graphs_python.php
Changes to include weighted edges from https://towardsdatascience.com/to-all-data-scientists-the-one-graph-algorithm-you-need-to-know-59178dbb1ec2
Some functions have been removed because they are not 
going to be used, and I would like to protect future 
users from using this graph object incorrectly.
"""

class Graph(object):
    def __init__(self, graph_dict=None):
        """ initializes a graph object 
            If no dictionary or None is given, 
            an empty dictionary will be used
        """
        if graph_dict == None:
            graph_dict = {}
        self.__graph_dict = graph_dict
        self.__vertices = self.vertices()

    def vertex_ids(self):
        """ returns the vertices of a graph """
        vertices = []
        for val in self.__graph_dict:
            vertices += [val['ID']]
        self.__vertices = vertices
        return list(self.__vertices)
    
    def vertices(self):
        """ returns the vertices of a graph """
        vertices = []
        for val in self.__graph_dict:
            ver = self.__removekey(val, 'edges')
            vertices += [ver]
        self.__vertices = vertices
        return list(self.__vertices)
    
    def get_vertex_from_vid(self, vid):
        ''' 
        returns the vertex given its id.
        I would LIKE to assume that the vertex's id will match its location in
        graph_dict, but I won't in the case that someone passes in an irregualar
        graph_dict to the Graph object.
        '''
        vertex = None
        for val in self.__graph_dict:
            if val['ID'] == vid:
                vertex = val
        if vertex == None:
            raise Exception('Vertex ID provided not found.')
        return vertex

    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()
    
    def get_edges_from_vid(self, vid):
        ''' 
        returns the edges from a vertex, given its id.
        '''
        vertex = self.get_vertex_from_vid(vid)
        edges = self.get_edges(vertex)
        return edges

    def __generate_edges(self):
        """ A static method generating the edges of the 
            graph "graph". Edges are represented as sets 
            with one (a loop back to the vertex) or two 
            vertices 
        """
        edges = []
        for vertex in self.__graph_dict:
            edges += self.get_edges(vertex)
        return edges

    def get_edges(self, vertex):
        edges = []
        for neighbour in vertex['edges']:
            weight = neighbour[1]
            neighbour = neighbour[0]
            v = vertex['ID']
            if (neighbour, vertex, weight) not in edges:
                edges.append([v, neighbour, weight])
        return edges
    
    def __str__(self):
        res = "vertices:\n"
        for k in self.__vertices:
            res += str(k) + " \n"
        res += "\nedges:\n"
        for edge in self.__generate_edges():
            res += str(edge) + " "
        return res

    def adj_mat(self):
        return self.__graph_dict
    
    def __removekey(self, d, key):
        r = dict(d)
        del r[key]
        return r

In [128]:
graph = Graph(vertices)
print(graph)

vertices:
{'ID': 0, 'start': 660.0, 'end': 840.0, 'address_hash': b'\xb0\xb2\x10xT,\x10\x84\xe9\xa7\xf31\xe4\x12\xad\xb8"N\x19\xf1'} 
{'ID': 1, 'start': 540.0, 'end': 660.0, 'address_hash': b'\x08\xfa{s@-G\xa0N5\xbf:e\xfc(\x004\x82\xa7#'} 
{'ID': 2, 'start': 720.0, 'end': 840.0, 'address_hash': b'`\x80a\xb0N\xf5\xbe!\xc2\xcd\xd6\xab\x86\xa9z\xb9\xf2\xc2\x86Z'} 
{'ID': 3, 'start': 600.0, 'end': 720.0, 'address_hash': b'\xe2s\xe0\x1aA\xd7;\xd2-\x05\xb4}{\x8d\xaa\tK\xfe\x06t'} 
{'ID': 4, 'start': 660.0, 'end': 780.0, 'address_hash': b'\xaa\x84qy\x08\x8f\x18)\xf1\x8b3\xa2R\xbde\xc2M6N\x87'} 

edges:
[0, 1, 40.45] [0, 2, 36.66] [0, 3, 25.56] [0, 4, 28.85] [1, 0, 41.73] [1, 2, 7.47] [1, 3, 19.05] [1, 4, 12.05] [2, 0, 36.25] [2, 1, 7.36] [2, 3, 19.4] [2, 4, 10.63] [3, 0, 25.0] [3, 1, 17.13] [3, 2, 18.81] [3, 4, 14.31] [4, 0, 27.43] [4, 1, 11.96] [4, 2, 9.52] [4, 3, 14.25] 


In [120]:
print(graph.get_edges_from_vid(0))

[[0, 1, 40.45], [0, 2, 36.66], [0, 3, 25.56], [0, 4, 28.85]]


In [181]:
def visit_next(graph, current_vertex, arrival_time, average_time_at_each_house = 30, visited = []):
    current_time = arrival_time
    idx = current_vertex['ID']
    print('Arrived at vertex %d at time %f' % (idx, arrival_time))
    visited += [idx]
    current_time += average_time_at_each_house
    
    print('Leaving at vertex %d at time %f' % (idx, current_time))
    
    edges = graph.get_edges(current_vertex)
    edges_no_cycle = get_acyclic_edges(edges, visited)
    print(edges_no_cycle)
    
    if len(edges_no_cycle) > 0:
        for edge in edges_no_cycle:
            next_vertex = graph.get_vertex_from_vid(edge[1])
            current_time_1 = current_time + edge[2]
            
            
            '''
            Check if the Open house has started yet, and wait for it to open
            if it hasn't closed yet.
            '''
            opened, closed = open_and_closed(current_time_1, next_vertex)
            wait_function = get_wait_function(opened, closed)
            current_time_2 = wait_function(next_vertex['start'], current_time_1)
            if current_time_2 < 0:
                continue
            else:
                return visit_next(graph, next_vertex, current_time_2, visited = visited)
    print(visited)
    print('---------------')
    hours, minutes = convert_mins_to_time(current_time)

    return visited, '%s:%s' %(hours, minutes)

def get_acyclic_edges(edges, visited):
    E = []
    for edge in edges:
        idx = edge[1]
        if been_visited(visited, idx):
            pass
        else:
            E += [edge]
    return E
        
def been_visited(visited, v):
    '''
    Checks to make sure you're not going back to a node that has already been visited.
    '''
    return v in visited

def open_and_closed(arrival_time, next_vertex):
    opened = arrival_time >= next_vertex['start']
    closed = arrival_time >= next_vertex['end']
    return [opened, closed]

def get_wait_function(opened, closed): #ideal True, True
    if opened and not closed:
        wait = "No need to wait"
    elif not opened and not closed:
        wait = "Wait"
    elif opened and closed:
        wait = "Too late"
    else:
        wait = "Time doesn't work that way!"
    wait_function = {
        "No need to wait" : lambda opens_at, current_time : current_time,
        "Wait" : lambda opens_at, current_time : opens_at,
        "Too late" : lambda opens_at, current_time : -1,
        "Time doesn't work that way!" : lambda opens_at, current_time : -1
    }
    return wait_function[wait]
    
def convert_mins_to_time(time):
    hours_minutes = str(time / 60).split('.')
    hours_minutes[0] = hours_minutes[0]
    hours_minutes[1] = str(float('0.'+hours_minutes[1]) * .6)[2:4]
    return hours_minutes[0], hours_minutes[1]

In [182]:
'''
For the sake of initial testing, I am going to start by going to the first house on the list (ID = 0).
Also, the starting time will be that starting time of the vertex we're starting with.
    A better starting time can be defined later (i.e. when routing is solved).
'''
starting_id = 0
starting_vertex = graph.get_vertex_from_vid(starting_id)
print(starting_vertex)
start_time = starting_vertex['start']

print(starting_vertex)
visit_next(graph, starting_vertex, start_time, visited=[])

{'ID': 0, 'start': 660.0, 'end': 840.0, 'edges': [[1, 40.45], [2, 36.66], [3, 25.56], [4, 28.85]], 'address_hash': b'\xb0\xb2\x10xT,\x10\x84\xe9\xa7\xf31\xe4\x12\xad\xb8"N\x19\xf1'}
{'ID': 0, 'start': 660.0, 'end': 840.0, 'edges': [[1, 40.45], [2, 36.66], [3, 25.56], [4, 28.85]], 'address_hash': b'\xb0\xb2\x10xT,\x10\x84\xe9\xa7\xf31\xe4\x12\xad\xb8"N\x19\xf1'}
Arrived at vertex 0 at time 660.000000
Leaving at vertex 0 at time 690.000000
[[0, 1, 40.45], [0, 2, 36.66], [0, 3, 25.56], [0, 4, 28.85]]
Arrived at vertex 2 at time 726.660000
Leaving at vertex 2 at time 756.660000
[[2, 1, 7.36], [2, 3, 19.4], [2, 4, 10.63]]
Arrived at vertex 4 at time 767.290000
Leaving at vertex 4 at time 797.290000
[[4, 1, 11.96], [4, 3, 14.25]]
[0, 2, 4]
---------------


([0, 2, 4], '13:17')

In [183]:
V = graph.vertices()

In [184]:
paths = []
for v in V:
    starting_id = v['ID']
    starting_vertex = graph.get_vertex_from_vid(starting_id)
    start_time = starting_vertex['start']
    paths += [visit_next(graph, starting_vertex, start_time, visited=[])]
print(paths)

Arrived at vertex 0 at time 660.000000
Leaving at vertex 0 at time 690.000000
[[0, 1, 40.45], [0, 2, 36.66], [0, 3, 25.56], [0, 4, 28.85]]
Arrived at vertex 2 at time 726.660000
Leaving at vertex 2 at time 756.660000
[[2, 1, 7.36], [2, 3, 19.4], [2, 4, 10.63]]
Arrived at vertex 4 at time 767.290000
Leaving at vertex 4 at time 797.290000
[[4, 1, 11.96], [4, 3, 14.25]]
[0, 2, 4]
---------------
Arrived at vertex 1 at time 540.000000
Leaving at vertex 1 at time 570.000000
[[1, 0, 41.73], [1, 2, 7.47], [1, 3, 19.05], [1, 4, 12.05]]
Arrived at vertex 0 at time 660.000000
Leaving at vertex 0 at time 690.000000
[[0, 2, 36.66], [0, 3, 25.56], [0, 4, 28.85]]
Arrived at vertex 2 at time 726.660000
Leaving at vertex 2 at time 756.660000
[[2, 3, 19.4], [2, 4, 10.63]]
Arrived at vertex 4 at time 767.290000
Leaving at vertex 4 at time 797.290000
[[4, 3, 14.25]]
[1, 0, 2, 4]
---------------
Arrived at vertex 2 at time 720.000000
Leaving at vertex 2 at time 750.000000
[[2, 0, 36.25], [2, 1, 7.36], [2,